## Loading the Dataset

In [1]:
import time
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_1f6f2e6280144b23bb3fd53cbdb85cdd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='H3RY-cRAHjH4W0Kbzwe1nQnJy4oM49dgBZOdz0R3AUwB',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_1f6f2e6280144b23bb3fd53cbdb85cdd.get_object(Bucket='qwerty28b58f4c38184cafbd941800e4d7f669',Key='NationalNames.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Id,Name,Year,Gender,Count
0,1,Mary,1880,F,7065
1,2,Anna,1880,F,2604
2,3,Emma,1880,F,2003
3,4,Elizabeth,1880,F,1939
4,5,Minnie,1880,F,1746


## Connecting Pandas Dataframe with SQL

In [10]:
time1 = time.time()
sqlContext = SQLContext(sc)
sdf = sqlContext.createDataFrame(df_data_1)
sdf.printSchema()
print("response time: %s seconds" % (time.time() - time1))

root
 |-- Id: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Count: long (nullable = true)

response time: 113.948674917 seconds


### Problem 3.1 - Total number of birth registered in a year

In [7]:
time2 = time.time()
sdf.groupby(['Year'])\
.agg({"Count": "Sum"})\
.sort("Year", ascending=True)\
.show()
print("response time: %s seconds" % (time.time() - time2))

+----+----------+
|Year|sum(Count)|
+----+----------+
|1880|    201484|
|1881|    192699|
|1882|    221538|
|1883|    216950|
|1884|    243467|
|1885|    240855|
|1886|    255319|
|1887|    247396|
|1888|    299480|
|1889|    288950|
|1890|    301402|
|1891|    286678|
|1892|    334383|
|1893|    325223|
|1894|    338694|
|1895|    351028|
|1896|    357490|
|1897|    346960|
|1898|    381463|
|1899|    339235|
+----+----------+
only showing top 20 rows

response time: 6.16431713104 seconds


### Problem 3.2 - Total number of births registered in a year by gender

In [6]:
time3 = time.time()
sdf.groupby(['Year', 'Gender'])\
.agg({"Count": "Sum"})\
.sort("Year", ascending=True)\
.show()
print("response time: %s seconds" % (time.time() - time3))

+----+------+----------+
|Year|Gender|sum(Count)|
+----+------+----------+
|1880|     M|    110491|
|1880|     F|     90993|
|1881|     M|    100745|
|1881|     F|     91954|
|1882|     M|    113688|
|1882|     F|    107850|
|1883|     M|    104629|
|1883|     F|    112321|
|1884|     M|    114445|
|1884|     F|    129022|
|1885|     F|    133055|
|1885|     M|    107800|
|1886|     M|    110784|
|1886|     F|    144535|
|1887|     F|    145982|
|1887|     M|    101414|
|1888|     M|    120853|
|1888|     F|    178627|
|1889|     F|    178366|
|1889|     M|    110584|
+----+------+----------+
only showing top 20 rows

response time: 5.94586610794 seconds


### Problem 3.3 - Input a year and populate top 5 most popular names registered that year 

In [9]:
time4 = time.time()
Year = raw_input("Enter the Year: ")
sdf.registerTempTable("abc")

queryString = "select Year, Name, sum(COUNT) from abc where Year like %s group by Name, Year  order by sum(count) desc LIMIT 5" % Year

abc = sqlContext.sql(queryString).show()
print("response time: %s seconds" % (time.time() - time4))

Enter the Year: 1880
+----+-------+----------+
|Year|   Name|sum(COUNT)|
+----+-------+----------+
|1880|   John|      9701|
|1880|William|      9562|
|1880|   Mary|      7092|
|1880|  James|      5949|
|1880|Charles|      5359|
+----+-------+----------+

response time: 8.98127889633 seconds


### Problem 3.4 - Input a child name and populate total number of birth registrations throughout the dataset for that name

In [8]:
time5 = time.time()
Name = raw_input("Enter your name: ")
sdf.registerTempTable("abc")

qwe = "SELECT Sum(Count) FROM abc WHERE Name like '%s'" % Name
abc = sqlContext.sql(qwe).show()
print("response time: %s seconds" % (time.time() - time5))

Enter your name: Helen
+----------+
|sum(Count)|
+----------+
|   1019006|
+----------+

response time: 8.50237298012 seconds
